In [ ]:
'''
This script processes data from SCOC's M187 incarcerated individual death forms into a table format (these contain
many of the same fields as the DOCCS forms, but have a slightly different format, which is why a separate pipeline is necessary).
It first classifies the forms by layout type using a Microsoft Azure AI Document Intelligence model, then, it extracts
fields from each form using a Document Intelligence model custom built for each layout type. Finally,
following a manual review of the extracted fields, it standardizes and cleans fields that need to be converted
to the appropriate types (e.g. float, date, time).

To run this code, perform the following steps:
1. Install the packages in the cell below on your local machine.
2. Insert your Azure blob key, URL, and container name for the M187s in the cell below. Also insert your Azure formrecognizer endpoint and key.
3. Upload the M187s as PDFs to your Azure blob container, using a separate file for each M187.
4. Create a document extraction model for the M187s. To match the field names used throughout this code pipeline,
use the following field names in your FormRecognizer model:

        "DECEASED_NAME", "MRB_NUMBER", "DIN", "NYSID", "FACILITY_NAME", "FACILITY_CODE_2D", "ORI", \
        "REPORT_DATE", "REPORTING_OFFICIAL_NAME", "HEIGHT", "WEIGHT", "RACE", "SEX", \
        "SENTENCE", "BIRTH_DATE", "SENTENCE_DATE", "ARREST_CHARGES", "DATE_ARREST", "DATE_CONVICTION", "CONVICTION_CHARGES", \
        "HOSPITAL_NAME", "CHIEF_ADMIN_OFFICER_NAME", "AMBULANCE_RESCUE_SQUAD_NAME", "DATETIME_ADMITTED", "DATE_OF_LAST_ADMISSION", \
        "DEATH_DATETIME", "DEATH_DATETIME_REPORTED", "TERMINAL_INCIDENT_LOCATION", "REPORTED_IMMEDIATE_CAUSE_OF_DEATH", "FACILITY_ADMINISTRATORS_REPORT_OF_DEATH_CIRCUMSTANCES", \
        "OFFICER_SUPERVISING_DEATH_LOCATION", "ASSIGNED_HOUSING_UNIT", "HOUSING_UNIT_TYPE", \
        "AUTOPSY_DATE", "AUTOPSY_TIME", "AUTOPSY_LOCATION", "MEDICAL_EXAMINER_CORONER_NAME", "AUTOPSY_PERFORMED_YES", "AUTOPSY_PERFORMED_NO", \
        "SUPERVISION_PRIOR_TO_INCIDENT", "SUBSTANCE_ABUSE_DRUG", "SUBSTANCE_ABUSE_ALCOHOL", "SUBSTANCE_ABUSE_UNKNOWN", "SUBSTANCE_ABUSE_NO", \
        "MEDICAL_TREATMENT", "PSYCH_TREATMENT", "NO_TREATMENT", "MEDICAL_CONTACT", "PSYCH_CONTACT", \
        "INTAKE_SCREENING_YES", "INTAKE_SCREENING_NO"

  Fill in the name of your model under # model IDs in the cell below.
'''

In [ ]:
# import libraries
import os
import re
import pandas as pd
import numpy as np
import json
import datetime as dt
import pyodbc
import time

from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

from azure.storage.blob import BlobClient
from azure.storage.blob import BlobServiceClient, ContentSettings
from azure.storage.blob import ContainerClient

# Azure account info
AZURE_BLOB_ACCOUNT_KEY = "[INSERT ACCOUNT KEY HERE]"
AZURE_BLOB_ACCOUNT_URL = "[INSERT ACCOUNT URL HERE]"
AZURE_FORM_RECOGNIZER_ENDPOINT = "[INSERT FORMRECOGNIZER ENDPOINT HERE]"
AZURE_FORM_RECOGNIZER_KEY = "[INSERT FORMRECOGNIZER KEY HERE]"
AZURE_BLOB_M187_CONTAINER = "[INSERT CONTAINER NAME HERE]"

document_analysis_client = DocumentAnalysisClient(
    endpoint=AZURE_FORM_RECOGNIZER_ENDPOINT, credential=AzureKeyCredential(AZURE_FORM_RECOGNIZER_KEY)
)

# model IDs
SCOC_MODEL_ID = "[INSERT MODEL NAME HERE]"

# variable lists
RAW_DATE_COL_LIST = ["REPORT_DATE", "BIRTH_DATE", "SENTENCE_DATE", "DATE_ARREST", "DATE_CONVICTION", "DATE_OF_LAST_ADMISSION", "AUTOPSY_DATE"]
RAW_TIME_COL_LIST = ["AUTOPSY_TIME"]
RAW_DATETIME_COL_LIST = ["DATETIME_ADMITTED"]

In [ ]:
# reads names of M187 forms to process
def read_file_names_from_blob_container():
   '''
    Returns the list of file names in an Azure blob container.
   '''

    # gen empty lists for filenames
    FILE_LIST = []

    # connect to M187 forms blob container
    container = ContainerClient(account_url=AZURE_BLOB_ACCOUNT_URL, container_name=AZURE_BLOB_M187_CONTAINER, credential=AZURE_BLOB_ACCOUNT_KEY)

    # make list of blobs in container
    blob_list = container.list_blobs()

    # organize file names of PDF files from blob container
    for blob in blob_list:
        if (blob.name).split('.')[-1].lower() == 'pdf':
            FILE_LIST.append(blob.name)

    return FILE_LIST

In [ ]:
def delete_blob_from_blob_container(blob_name):
    '''
    Deletes an Azure blob from Azure blob container.
    '''

    # connect to M187 blob container
    container = ContainerClient(account_url=AZURE_BLOB_ACCOUNT_URL, container_name=AZURE_BLOB_M187_CONTAINER,credential=AZURE_BLOB_ACCOUNT_KEY)

    # delete blob
    container.delete_blob(blob=blob_name)

    return

In [ ]:
def save_csv_to_blob_container(df, file_name):
    '''
    Saves a Pandas DataFrame to an Azure blob container.
    '''

    # create the BlobServiceClient object
    blob_service_client = BlobServiceClient(AZURE_BLOB_ACCOUNT_URL, credential=AZURE_BLOB_ACCOUNT_KEY)
    blob_client = blob_service_client.get_blob_client(container=AZURE_BLOB_M187_CONTAINER, blob=file_name)
    blob_settings = ContentSettings(content_encoding='UTF-8')

    # save file as blob to container, cool storage
    blob_client.upload_blob(df.to_csv(encoding='UTF-8', index=False, sep='|'),overwrite=True,content_type="text/csv")
    blob_client.set_standard_blob_tier('Cool')

    return

In [ ]:
'''
The following functions  extract fields from M187s using your custom Azure Document Intelligence model.
'''

In [ ]:
def extract_info_from_scoc_form(file, fields_values_df):
    '''
    Uses a custom Azure Document Intelligence model to extract fields from an individual form.
    '''

    model_id = os.getenv("SCOC_MODEL_ID", SCOC_MODEL_ID)
    blob_client = BlobClient(account_url=AZURE_BLOB_ACCOUNT_URL, container_name="m187-scoc-forms-blob", blob_name=file, credential=AZURE_BLOB_ACCOUNT_KEY)

    poller = document_analysis_client.begin_analyze_document_from_url(
             model_id = SCOC_MODEL_ID, document_url=blob_client.url
         )
    result = poller.result()

    key_value_dict = {}
    key_confidence_dict = {}

    for document in result.documents:
        key_value_dict["FORM_TYPE"] = "SCOC"
        key_value_dict["FORM_URL"] = blob_client.url
        for field_name, field in document.fields.items():
                key_value_dict[field_name] = field.value
                key_confidence_dict[field_name] = field.confidence
        key_value_dict["FIELD_CONFIDENCE_DICT"] = key_confidence_dict
        temp_df = pd.DataFrame([key_value_dict])
        fields_values_df = pd.concat([fields_values_df, temp_df], ignore_index=True)

    return fields_values_df

In [ ]:
def extract_info_from_m187_forms(file_list, only_process_new_files, current_fields_values_df=None):
    '''
    Uses extract_info_from_scoc_form() to extract fields from all files in file list. Returns dataframe with extracted
    fields. Note that current_fields_values_df will be set to existing dataframe only if only_process_new_files = True.
    '''

    # if we want to process all files, create a new empty dataframe for fields and values
    if only_process_new_files == False:
        fields_values_df = pd.DataFrame(columns = ["POTENTIAL_DUPLICATE", "FILE_NAME", "FORM_TYPE", "FORM_URL", "MANUAL_REVIEW_DONE", "DECEASED_NAME", "MRB_NUMBER", "DIN", "NYSID", "FACILITY_NAME", "FACILITY_CODE_2D", "ORI", \
        "REPORT_DATE", "REPORTING_OFFICIAL_NAME", "HEIGHT", "WEIGHT", "RACE", "SEX", \
        "SENTENCE", "BIRTH_DATE", "SENTENCE_DATE", "ARREST_CHARGES", "DATE_ARREST", "DATE_CONVICTION", "CONVICTION_CHARGES", \
        "HOSPITAL_NAME", "CHIEF_ADMIN_OFFICER_NAME", "AMBULANCE_RESCUE_SQUAD_NAME", "DATETIME_ADMITTED", "DATE_OF_LAST_ADMISSION", \
        "DEATH_DATETIME", "DEATH_DATETIME_REPORTED", "TERMINAL_INCIDENT_LOCATION", "REPORTED_IMMEDIATE_CAUSE_OF_DEATH", "FACILITY_ADMINISTRATORS_REPORT_OF_DEATH_CIRCUMSTANCES", \
        "OFFICER_SUPERVISING_DEATH_LOCATION", "ASSIGNED_HOUSING_UNIT", "HOUSING_UNIT_TYPE", \
        "AUTOPSY_DATE", "AUTOPSY_TIME", "AUTOPSY_LOCATION", "MEDICAL_EXAMINER_CORONER_NAME", "AUTOPSY_PERFORMED_YES", "AUTOPSY_PERFORMED_NO", \
        "SUPERVISION_PRIOR_TO_INCIDENT", "SUBSTANCE_ABUSE_DRUG", "SUBSTANCE_ABUSE_ALCOHOL", "SUBSTANCE_ABUSE_UNKNOWN", "SUBSTANCE_ABUSE_NO", \
        "MEDICAL_TREATMENT", "PSYCH_TREATMENT", "NO_TREATMENT", "MEDICAL_CONTACT", "PSYCH_CONTACT", \
        "INTAKE_SCREENING_YES", "INTAKE_SCREENING_NO", "FIELD_CONFIDENCE_DICT"])

    # if we only want to process new values, set fields_values_df to current dataframe so we can append to it
    elif only_process_new_files == True:
        fields_values_df = current_fields_values_df

    # run field extraction models on each forms
    for file in file_list:
        fields_values_df = extract_info_from_scoc_form(file, fields_values_df)

    return fields_values_df


In [ ]:
'''
The following functions standardize and clean the data once it has been extracted using the Document Intelligence model.
They are called once prior to the manual review, and once afterwards.
'''

In [ ]:
def clean_doccs_facility_name(df):
    df.loc[df['FACILITY_NAME'].str.contains('ADIRONDACK'), 'FACILITY_NAME'] = 'ADIRONDACK'
    df.loc[df['FACILITY_NAME'].str.contains('ALBION'), 'FACILITY_NAME'] = 'ALBION'
    df.loc[df['FACILITY_NAME'].str.contains('ALTONA'), 'FACILITY_NAME'] = 'ALTONA'
    df.loc[df['FACILITY_NAME'].str.contains('ARTHUR KILL'), 'FACILITY_NAME'] = 'ARTHUR KILL'
    df.loc[df['FACILITY_NAME'].str.contains('ATTICA'), 'FACILITY_NAME'] = 'ATTICA'
    df.loc[df['FACILITY_NAME'].str.contains('AUBURN'), 'FACILITY_NAME'] = 'AUBURN'
    df.loc[df['FACILITY_NAME'].str.contains('BAYVIEW'), 'FACILITY_NAME'] = 'BAYVIEW'
    df.loc[df['FACILITY_NAME'].str.contains('BARE HILL'), 'FACILITY_NAME'] = 'BARE HILL'
    df.loc[df['FACILITY_NAME'].str.contains('BEACON'), 'FACILITY_NAME'] = 'BEACON'
    df.loc[df['FACILITY_NAME'].str.contains('BEDFORD HILLS'), 'FACILITY_NAME'] = 'BEDFORD HILLS'
    df.loc[df['FACILITY_NAME'].str.contains('BUFFALO'), 'FACILITY_NAME'] = 'BUFFALO'
    df.loc[df['FACILITY_NAME'].str.contains('BUTLER'), 'FACILITY_NAME'] = 'BUTLER'
    df.loc[df['FACILITY_NAME'].str.contains('CAMP GABRIELS'), 'FACILITY_NAME'] = 'CAMP GABRIELS'
    df.loc[df['FACILITY_NAME'].str.contains('CAMP GEORGETOWN'), 'FACILITY_NAME'] = 'CAMP GEORGETOWN'
    df.loc[df['FACILITY_NAME'].str.contains('CAMP PHARSALIA'), 'FACILITY_NAME'] = 'CAMP PHARSALIA'
    df.loc[df['FACILITY_NAME'].str.contains('CAPE VINCENT'), 'FACILITY_NAME'] = 'CAPE VINCENT'
    df.loc[df['FACILITY_NAME'].str.contains('CAYUGA'), 'FACILITY_NAME'] = 'CAYUGA'
    df.loc[df['FACILITY_NAME'].str.contains('CHATEAUGAY'), 'FACILITY_NAME'] = 'CHATEAUGAY'
    df.loc[df['FACILITY_NAME'].str.contains('CLINTON'), 'FACILITY_NAME'] = 'CLINTON'
    df.loc[df['FACILITY_NAME'].str.contains('COLLINS'), 'FACILITY_NAME'] = 'COLLINS'
    df.loc[df['FACILITY_NAME'].str.contains('COXSACKIE'), 'FACILITY_NAME'] = 'COXSACKIE'
    df.loc[df['FACILITY_NAME'].str.contains('DOWNSTATE'), 'FACILITY_NAME'] = 'DOWNSTATE'
    df.loc[df['FACILITY_NAME'].str.contains('EASTERN'), 'FACILITY_NAME'] = 'EASTERN'
    df.loc[df['FACILITY_NAME'].str.contains('EDGECOMBE'), 'FACILITY_NAME'] = 'EDGECOMBE'
    df.loc[df['FACILITY_NAME'].str.contains('ELMIRA'), 'FACILITY_NAME'] = 'ELMIRA'
    df.loc[df['FACILITY_NAME'].str.contains('FISHKILL'), 'FACILITY_NAME'] = 'FISHKILL'
    df.loc[df['FACILITY_NAME'].str.contains('FIVE POINTS'), 'FACILITY_NAME'] = 'FIVE POINTS'
    df.loc[df['FACILITY_NAME'].str.contains('FRANKLIN'), 'FACILITY_NAME'] = 'FRANKLIN'
    df.loc[df['FACILITY_NAME'].str.contains('FULTON'), 'FACILITY_NAME'] = 'FULTON'
    df.loc[df['FACILITY_NAME'].str.contains('GOWANDA'), 'FACILITY_NAME'] = 'GOWANDA'
    df.loc[df['FACILITY_NAME'].str.contains('GOUVERNEUR'), 'FACILITY_NAME'] = 'GOUVERNEUR'
    df.loc[df['FACILITY_NAME'].str.contains('GREAT MEADOW'), 'FACILITY_NAME'] = 'GREAT MEADOW'
    df.loc[df['FACILITY_NAME'].str.contains('GREEN HAVEN'), 'FACILITY_NAME'] = 'GREEN HAVEN'
    df.loc[df['FACILITY_NAME'].str.contains('GREENE'), 'FACILITY_NAME'] = 'GREENE'
    df.loc[df['FACILITY_NAME'].str.contains('GROVELAND'), 'FACILITY_NAME'] = 'GROVELAND'
    df.loc[df['FACILITY_NAME'].str.contains('HALE CREEK'), 'FACILITY_NAME'] = 'HALE CREEK'
    df.loc[df['FACILITY_NAME'].str.contains('HUDSON'), 'FACILITY_NAME'] = 'HUDSON'
    df.loc[df['FACILITY_NAME'].str.contains('LAKEVIEW'), 'FACILITY_NAME'] = 'LAKEVIEW'
    df.loc[df['FACILITY_NAME'].str.contains('LINCOLN'), 'FACILITY_NAME'] = 'LINCOLN'
    df.loc[df['FACILITY_NAME'].str.contains('LIVINGSTON'), 'FACILITY_NAME'] = 'LIVINGSTON'
    df.loc[df['FACILITY_NAME'].str.contains('LYON MOUNTAIN'), 'FACILITY_NAME'] = 'LYON MOUNTAIN'
    df.loc[df['FACILITY_NAME'].str.contains('MARCY'), 'FACILITY_NAME'] = 'MARCY'
    df.loc[df['FACILITY_NAME'].str.contains('MID-ORANGE'), 'FACILITY_NAME'] = 'MID-ORANGE'
    df.loc[df['FACILITY_NAME'].str.contains('MID-STATE'), 'FACILITY_NAME'] = 'MID-STATE'
    df.loc[df['FACILITY_NAME'].str.contains('MOHAWK'), 'FACILITY_NAME'] = 'MOHAWK'
    df.loc[df['FACILITY_NAME'].str.contains('MONTEREY'), 'FACILITY_NAME'] = 'MONTEREY'
    df.loc[df['FACILITY_NAME'].str.contains('MORIAH'), 'FACILITY_NAME'] = 'MORIAH'
    df.loc[df['FACILITY_NAME'].str.contains('MOUNT MCGREGOR'), 'FACILITY_NAME'] = 'MOUNT MCGREGOR'
    df.loc[df['FACILITY_NAME'].str.contains('OGDENSBURG'), 'FACILITY_NAME'] = 'OGDENSBURG'
    df.loc[df['FACILITY_NAME'].str.contains('ONEIDA'), 'FACILITY_NAME'] = 'ONEIDA'
    df.loc[df['FACILITY_NAME'].str.contains('ORLEANS'), 'FACILITY_NAME'] = 'ORLEANS'
    df.loc[df['FACILITY_NAME'].str.contains('OTISVILLE'), 'FACILITY_NAME'] = 'OTISVILLE'
    df.loc[df['FACILITY_NAME'].str.contains('QUEENSBORO'), 'FACILITY_NAME'] = 'QUEENSBORO'
    df.loc[df['FACILITY_NAME'].str.contains('RIVERVIEW'), 'FACILITY_NAME'] = 'RIVERVIEW'
    df.loc[df['FACILITY_NAME'].str.contains('ROCHESTER'), 'FACILITY_NAME'] = 'ROCHESTER'
    df.loc[df['FACILITY_NAME'].str.contains('SHAWANGUNK'), 'FACILITY_NAME'] = 'SHAWANGUNK'
    df.loc[df['FACILITY_NAME'].str.contains('SING SING'), 'FACILITY_NAME'] = 'SING SING'
    df.loc[df['FACILITY_NAME'].str.contains('SOUTHPORT'), 'FACILITY_NAME'] = 'SOUTHPORT'
    df.loc[df['FACILITY_NAME'].str.contains('SULLIVAN'), 'FACILITY_NAME'] = 'SULLIVAN'
    df.loc[df['FACILITY_NAME'].str.contains('SUMMIT'), 'FACILITY_NAME'] = 'SUMMIT'
    df.loc[df['FACILITY_NAME'].str.contains('TACONIC'), 'FACILITY_NAME'] = 'TACONIC'
    df.loc[df['FACILITY_NAME'].str.contains('ULSTER'), 'FACILITY_NAME'] = 'ULSTER'
    df.loc[df['FACILITY_NAME'].str.contains('UPSTATE'), 'FACILITY_NAME'] = 'UPSTATE'
    df.loc[df['FACILITY_NAME'].str.contains('WALSH'), 'FACILITY_NAME'] = 'WALSH'
    df.loc[df['FACILITY_NAME'].str.contains('WALLKILL'), 'FACILITY_NAME'] = 'WALLKILL'
    df.loc[df['FACILITY_NAME'].str.contains('WATERTOWN'), 'FACILITY_NAME'] = 'WATERTOWN'
    df.loc[df['FACILITY_NAME'].str.contains('WASHINGTON'), 'FACILITY_NAME'] = 'WASHINGTON'
    df.loc[df['FACILITY_NAME'].str.contains('WENDE'), 'FACILITY_NAME'] = 'WENDE'
    df.loc[df['FACILITY_NAME'].str.contains('WILLARD'), 'FACILITY_NAME'] = 'WILLARD'
    df.loc[df['FACILITY_NAME'].str.contains('WOODBOURNE'), 'FACILITY_NAME'] = 'WOODBOURNE'
    df.loc[df['FACILITY_NAME'].str.contains('WYOMING'), 'FACILITY_NAME'] = 'WYOMING'

    return df

In [ ]:
def create_facility_code_2d(df):
    df.loc[df['FACILITY_NAME'].str.contains('ADIRONDACK'), 'FACILITY_CODE_2D'] = '23'
    df.loc[df['FACILITY_NAME'].str.contains('ALBION'), 'FACILITY_CODE_2D'] = '09'
    df.loc[df['FACILITY_NAME'].str.contains('ALTONA'), 'FACILITY_CODE_2D'] = '54'
    df.loc[df['FACILITY_NAME'].str.contains('ARTHUR KILL'), 'FACILITY_CODE_2D'] = '15'
    df.loc[df['FACILITY_NAME'].str.contains('ATTICA'), 'FACILITY_CODE_2D'] = '00'
    df.loc[df['FACILITY_NAME'].str.contains('AUBURN'), 'FACILITY_CODE_2D'] = '01'
    df.loc[df['FACILITY_NAME'].str.contains('BARE HILL'), 'FACILITY_CODE_2D'] = '56'
    df.loc[df['FACILITY_NAME'].str.contains('BAYVIEW'), 'FACILITY_CODE_2D'] = '31'
    df.loc[df['FACILITY_NAME'].str.contains('BEACON'), 'FACILITY_CODE_2D'] = '34'
    df.loc[df['FACILITY_NAME'].str.contains('BEDFORD HILLS'), 'FACILITY_CODE_2D'] = '12'
    df.loc[df['FACILITY_NAME'].str.contains('BUFFALO'), 'FACILITY_CODE_2D'] = '88'
    df.loc[df['FACILITY_NAME'].str.contains('BUTLER'), 'FACILITY_CODE_2D'] = '52'
    df.loc[df['FACILITY_NAME'].str.contains('CAMP GABRIELS'), 'FACILITY_CODE_2D'] = '22'
    df.loc[df['FACILITY_NAME'].str.contains('CAMP GEORGETOWN'), 'FACILITY_CODE_2D'] = '21'
    df.loc[df['FACILITY_NAME'].str.contains('CAMP PHARSALIA'), 'FACILITY_CODE_2D'] = '18'
    df.loc[df['FACILITY_NAME'].str.contains('CAPE VINCENT'), 'FACILITY_CODE_2D'] = '58'
    df.loc[df['FACILITY_NAME'].str.contains('CAYUGA'), 'FACILITY_CODE_2D'] = '55'
    df.loc[df['FACILITY_NAME'].str.contains('CHATEAUGAY'), 'FACILITY_CODE_2D'] = '86'
    df.loc[df['FACILITY_NAME'].str.contains('CLINTON'), 'FACILITY_CODE_2D'] = '02'
    df.loc[df['FACILITY_NAME'].str.contains('COLLINS'), 'FACILITY_CODE_2D'] = '47'
    df.loc[df['FACILITY_NAME'].str.contains('COXSACKIE'), 'FACILITY_CODE_2D'] = '13'
    df.loc[df['FACILITY_NAME'].str.contains('DOWNSTATE'), 'FACILITY_CODE_2D'] = '24'
    df.loc[df['FACILITY_NAME'].str.contains('EASTERN'), 'FACILITY_CODE_2D'] = '10'
    df.loc[df['FACILITY_NAME'].str.contains('EDGECOMBE'), 'FACILITY_CODE_2D'] = '32'
    df.loc[df['FACILITY_NAME'].str.contains('ELMIRA'), 'FACILITY_CODE_2D'] = '11'
    df.loc[df['FACILITY_NAME'].str.contains('FISHKILL'), 'FACILITY_CODE_2D'] = '05'
    df.loc[df['FACILITY_NAME'].str.contains('FIVE POINT'), 'FACILITY_CODE_2D'] = '37'
    df.loc[df['FACILITY_NAME'].str.contains('FRANKLIN'), 'FACILITY_CODE_2D'] = '53'
    df.loc[df['FACILITY_NAME'].str.contains('FULTON'), 'FACILITY_CODE_2D'] = '38'
    df.loc[df['FACILITY_NAME'].str.contains('GOWANDA'), 'FACILITY_CODE_2D'] = '45'
    df.loc[df['FACILITY_NAME'].str.contains('GOUVERNEUR'), 'FACILITY_CODE_2D'] = '81'
    df.loc[df['FACILITY_NAME'].str.contains('GREAT MEADOW'), 'FACILITY_CODE_2D'] = '04'
    df.loc[df['FACILITY_NAME'].str.contains('GREEN HAVEN'), 'FACILITY_CODE_2D'] = '08'
    df.loc[df['FACILITY_NAME'].str.contains('GREENE'), 'FACILITY_CODE_2D'] = '67'
    df.loc[df['FACILITY_NAME'].str.contains('GROVELAND'), 'FACILITY_CODE_2D'] = '46'
    df.loc[df['FACILITY_NAME'].str.contains('HALE CREEK'), 'FACILITY_CODE_2D'] = '85'
    df.loc[df['FACILITY_NAME'].str.contains('HUDSON'), 'FACILITY_CODE_2D'] = '27'
    df.loc[df['FACILITY_NAME'].str.contains('LAKEVIEW'), 'FACILITY_CODE_2D'] = '60'
    df.loc[df['FACILITY_NAME'].str.contains('LINCOLN'), 'FACILITY_CODE_2D'] = '36'
    df.loc[df['FACILITY_NAME'].str.contains('LIVINGSTON'), 'FACILITY_CODE_2D'] = '80'
    df.loc[df['FACILITY_NAME'].str.contains('LYON MOUNTAIN'), 'FACILITY_CODE_2D'] = '59'
    df.loc[df['FACILITY_NAME'].str.contains('MARCY'), 'FACILITY_CODE_2D'] = '49'
    df.loc[df['FACILITY_NAME'].str.contains('MID-ORANGE'), 'FACILITY_CODE_2D'] = '28'
    df.loc[(df['FACILITY_NAME'].str.contains('MIDSTATE'))|(df['FACILITY_NAME'].str.contains('MID-STATE')), 'FACILITY_CODE_2D'] = '48'
    df.loc[df['FACILITY_NAME'].str.contains('MOHAWK'), 'FACILITY_CODE_2D'] = '39'
    df.loc[df['FACILITY_NAME'].str.contains('WALSH'), 'FACILITY_CODE_2D'] = '39'  # make Walsh have same facility code as Mohawk
    df.loc[df['FACILITY_NAME'].str.contains('MONTEREY'), 'FACILITY_CODE_2D'] = '19'
    df.loc[df['FACILITY_NAME'].str.contains('MORIAH'), 'FACILITY_CODE_2D'] = '51'
    df.loc[df['FACILITY_NAME'].str.contains('MCGREGOR'), 'FACILITY_CODE_2D'] = '26'
    df.loc[df['FACILITY_NAME'].str.contains('OGDENSBURG'), 'FACILITY_CODE_2D'] = '35'
    df.loc[df['FACILITY_NAME'].str.contains('ONEIDA'), 'FACILITY_CODE_2D'] = '44'
    df.loc[df['FACILITY_NAME'].str.contains('ORLEANS'), 'FACILITY_CODE_2D'] = '64'
    df.loc[df['FACILITY_NAME'].str.contains('OTISVILLE'), 'FACILITY_CODE_2D'] = '29'
    df.loc[df['FACILITY_NAME'].str.contains('QUEENSBORO'), 'FACILITY_CODE_2D'] = '17'
    df.loc[df['FACILITY_NAME'].str.contains('RIVERVIEW'), 'FACILITY_CODE_2D'] = '57'
    df.loc[df['FACILITY_NAME'].str.contains('ROCHESTER'), 'FACILITY_CODE_2D'] = '30'
    df.loc[df['FACILITY_NAME'].str.contains('SHAWANGUNK'), 'FACILITY_CODE_2D'] = '68'
    df.loc[df['FACILITY_NAME'].str.contains('SING SING'), 'FACILITY_CODE_2D'] = '07'
    df.loc[df['FACILITY_NAME'].str.contains('SOUTHPORT'), 'FACILITY_CODE_2D'] = '63'
    df.loc[df['FACILITY_NAME'].str.contains('SULLIVAN'), 'FACILITY_CODE_2D'] = '69'
    df.loc[df['FACILITY_NAME'].str.contains('SUMMIT'), 'FACILITY_CODE_2D'] = '20'
    df.loc[df['FACILITY_NAME'].str.contains('TACONIC'), 'FACILITY_CODE_2D'] = '25'
    df.loc[df['FACILITY_NAME'].str.contains('ULSTER'), 'FACILITY_CODE_2D'] = '61'
    df.loc[df['FACILITY_NAME'].str.contains('UPSTATE'), 'FACILITY_CODE_2D'] = '84'
    df.loc[df['FACILITY_NAME'].str.contains('WALLKILL'), 'FACILITY_CODE_2D'] = '06'
    df.loc[df['FACILITY_NAME'].str.contains('WATERTOWN'), 'FACILITY_CODE_2D'] = '03'
    df.loc[df['FACILITY_NAME'].str.contains('WASHINGTON'), 'FACILITY_CODE_2D'] = '65'
    df.loc[df['FACILITY_NAME'].str.contains('WENDE'), 'FACILITY_CODE_2D'] = '43'
    df.loc[df['FACILITY_NAME'].str.contains('WILLARD'), 'FACILITY_CODE_2D'] = '82'
    df.loc[df['FACILITY_NAME'].str.contains('WOODBOURNE'), 'FACILITY_CODE_2D'] = '14'
    df.loc[df['FACILITY_NAME'].str.contains('WYOMING'), 'FACILITY_CODE_2D'] = '66'

    return df

In [ ]:
def to_upper(df):
    df = df.applymap(lambda s: s.upper() if type(s) == str else s)

    return df

In [ ]:
def clean_race_col(df):

    df["RACE"] = df["RACE"].replace("A", "ASIAN")
    df["RACE"] = df["RACE"].replace("B", "BLACK")
    df["RACE"] = df["RACE"].replace("W", "WHITE")
    df["RACE"] = df["RACE"].replace("O", "OTHER")

    return df

In [ ]:
def clean_date_time_cols(df):
    for col in RAW_DATE_COL_LIST:
        df[col] = pd.to_datetime(df[col],  errors = 'ignore')

    for col in RAW_TIME_COL_LIST:
        df[col] = pd.to_datetime(df[col], errors = 'ignore')

    for col in RAW_DATETIME_COL_LIST:
        df[col] = pd.to_datetime(df[col], errors = 'ignore')

    return df

In [ ]:
def clean_remaining_cols(df):

    # sentence
    df["SENTENCE"] = df["SENTENCE"].str.replace("YRS", "YEARS", regex = True)
    df["SENTENCE"] = df["SENTENCE"].str.replace(";", ", ", regex = True)

    # housing unit abbreviations
    df = df.replace({'REGIONAL MEDICAL UNIT': 'RMU'}, regex=True)
    df = df.replace({'GP': 'GENERAL POPULATION'}, regex=False)
    df.loc[df['HOUSING_UNIT_TYPE'].str.contains('GEN'), 'HOUSING_UNIT_TYPE'] = 'GENERAL POPULATION'

    # form URL -- make the appropriate phrases lowercase so the PDF will download properly
    df["FORM_URL"] = df["FORM_URL"].str.replace(".PDF", ".pdf", regex = True)
    df["FORM_URL"] = df["FORM_URL"].str.replace("M187-SCOC-FORMS-BLOB", "m187-scoc-forms-blob", regex = True)

    return df

In [ ]:
def strip_whitespace(df):
    df = df.applymap(lambda s: s.strip() if type(s) == str else s)

    return df

In [ ]:
def clean_df(df):
    '''
    Runs all data cleaning and standardization functions.
    '''
    df = df.astype(str)
    df = df.dropna(how='all')
    df = df.replace({'nan': ''}, regex=True)
    df = df.replace({'|': ''}, regex=True)
    df = df.replace({'\r': ''}, regex=True)
    df = df.replace({'DNA': 'DID NOT ANSWER'}, regex=True)

    df = to_upper(df)
    df = clean_doccs_facility_name(df)
    df = clean_race_col(df)
    df = clean_date_time_cols(df)
    df = clean_remaining_cols(df)
    df = strip_whitespace(df)

    df = df.astype(str)
    df = df.replace({'NaT': ''}, regex=True)
    df = df.replace({'NONE': ''}, regex=True)

    # abbreviate checkbox indicators for ease of checking
    df = df.replace({'UNSELECTED': ''}, regex=True)
    df = df.replace({'SELECTED': 'X'}, regex=True)

    return df

In [ ]:
def process_raw_forms(only_process_new_files):
    '''
    The main function for before the manual review step. Classifies and extracts information from forms and performs
    preliminary data cleaning before uploading to the Azure blob as a CSV. Set only_process_new_files to True if we
    only want to process M187s that haven't been processed yet; set to False if we want to drop all processed data
    and reprocess all forms.
    '''

    # read file names from blob container
    FILE_LIST = read_file_names_from_blob_container()

    # if we only want to process new files, update list only with files which don't already have their fields in the data table
    if only_process_new_files == True:
        # get CSV with fields extracted from forms so far and turn it into a dataframe so that we can add to it
        blob_client = BlobClient(account_url=AZURE_BLOB_ACCOUNT_URL, container_name=AZURE_BLOB_M187_CONTAINER, blob_name="M187_SCOC_extracted_fields_table.csv", credential=AZURE_BLOB_ACCOUNT_KEY)
        blob_download = blob_client.download_blob()
        blob_content = blob_download.readall().decode('UTF-8')
        M187_extracted_fields_df = pd.DataFrame([x.split('|') for x in blob_content.split('\n')])
        M187_extracted_fields_df = M187_extracted_fields_df.replace({'"FIELD_CONFIDENCE_DICT\r"\r': 'FIELD_CONFIDENCE_DICT'}, regex=True)

        # delete old extracted fields df so we can upload a new one later
        delete_blob_from_blob_container("M187_SCOC_extracted_fields_table.csv")

        # make top row header
        header = M187_extracted_fields_df.iloc[0]
        M187_extracted_fields_df = M187_extracted_fields_df[1:]
        M187_extracted_fields_df.columns = header

        processed_file_list = M187_extracted_fields_df["FILE_NAME"].tolist()
        processed_file_list = [f for f in processed_file_list if f is not None]
        processed_file_list = [f.replace("PDF", "pdf") for f in processed_file_list]
        processed_file_list = [f.replace("PAGES", "pages") for f in processed_file_list]

        # drop names of processed files (i.e. files already represented in the extracted fields dataframe, even if they haven't been manually checked yet) from the list
        FILE_LIST = [f for f in FILE_LIST if f not in processed_file_list]

        print('Files to process:', FILE_LIST)
        df = extract_info_from_m187_forms(FILE_LIST, only_process_new_files, current_fields_values_df = M187_extracted_fields_df)

    # if we want to process all files, we can ignore the extracted fields dataframe already in the blob
    elif only_process_new_files == False:
        df["MANUAL_REVIEW_DONE"] = "NO"
        print('Files to process:', FILE_LIST)
        df = extract_info_from_m187_forms(FILE_LIST, only_process_new_files)

    df = clean_df(df)
    save_csv_to_blob_container(df, "M187_SCOC_extracted_fields_table.csv")
    print('Processing complete')

    return df

In [ ]:
def clean_df_post_manual_check():
    '''
    The main function for after the manual review step. Performs data QA and final data cleaning before uploading the
    cleaned file to the Azure blob as a CSV.
    '''
    file = "M187_SCOC_extracted_fields_table.csv"
    blob_client = BlobClient(account_url=AZURE_BLOB_ACCOUNT_URL, container_name="m187-scoc-forms-blob", blob_name=file, credential=AZURE_BLOB_ACCOUNT_KEY)
    blob_download = blob_client.download_blob()
    blob_content = blob_download.readall().decode('UTF-8')
    df = pd.DataFrame([x.split('|') for x in blob_content.split('\n')])

    # make top row header
    header = df.iloc[0]
    df = df[1:]
    df.columns = header

    # final clean of dataframe
    df = clean_df(df)

    # check DIN and set to missing if necessary
    df.loc[df["DIN"].apply(lambda x: not re.match("^\d{2}[A-Z]\d{4}$", x)), "DIN"] = ""

    # add facility code
    df = create_facility_code_2d(df)

    # delete the old blob from the container so we can upload the new dataframe
    delete_blob_from_blob_container("M187_SCOC_extracted_fields_table.csv")

    # save cleaned dataframe to container
    save_csv_to_blob_container(df, "M187_SCOC_extracted_fields_table.csv")

    return df

In [ ]:
# main function execution
def main(process_data, only_process_new_files, run_post_processing):

    if process_data == True:

        process_raw_forms(only_process_new_files)


    if run_post_processing == True:

        clean_data_post_processing()

    return

In [ ]:
# run settings
'''
First set process_data to True and run_post_manual_review to False. Set only_process_new_files to either True or False depending on
whether to process only new files, or to reprocess all files.

Then, perform the MANUAL REVIEW STEP: Manually clean the output data to correct Document Intelligence errors. Flag redactions and keep only one
duplicate for each set of duplicates.

Finally, set process_data to False and run_post_manual_review to True. This will process the manually reviewed data to standardize the formatting.
'''
process_data = True # read and transform raw data files

run_post_manual_review = False # run data processing after manual review process has been completed

main(process_data, only_process_new_files, run_post_processing)